# ハイパーパラメーターの調整

*ハイパーパラメーター*（トレーニングに影響を与えるが、トレーニングデータ自体から決定できないパラメーター値）を必要とする多くの機械学習アルゴリズムがあります。たとえば、ロジスティック回帰モデルをトレーニングする場合、* regularization rate *ハイパーパラメーターを使用してモデルのバイアスに対抗できます。または畳み込みニューラルネットワークをトレーニングする場合、*学習率*や*バッチサイズ*などのハイパーパラメーターを使用して、重みの調整方法とミニバッチで処理されるデータ項目の数をそれぞれ制御できます。ハイパーパラメーター値の選択は、トレーニング済みモデルのパフォーマンス、またはトレーニングにかかる​​時間に大きく影響する可能性があります。多くの場合、最適なソリューションを見つけるために複数の組み合わせを試す必要があります。

この場合、単一のハイパーパラメーターを持つロジスティック回帰モデルの簡単な例を使用しますが、原則はAzure Machine Learningでトレーニングできるあらゆる種類のモデルに適用されます。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用のデータを準備する

このラボでは、糖尿病患者の詳細を含むデータセットを使用します。以下のセルを実行して、このデータセットを作成します（以前のラボで作成した場合、コードは新しいバージョンを作成します）

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Register the tabular dataset
tab_data_set = tab_data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Dataset ready.')

## トレーニングスクリプトを準備する

まず、ロジスティック回帰モデルのトレーニングに使用するトレーニングスクリプト用のフォルダーを作成することから始めましょう。

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

次に、モデルをトレーニングするPythonスクリプトを作成します。これには:

- 最適化する各ハイパーパラメーターのパラメーター（この場合、正則化ハイパーパラメーターのみがあります）
- 最適化するパフォーマンスメトリックを記録するコード（この場合、AUCと精度の両方を記録するため、これらのいずれかに対してモデルを最適化することを選択できます）

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
import joblib
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Set regularization parameter
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe() # Get the training data from the estimator input

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 計算ターゲットを準備する

クラウドコンピューティングの利点の1つは、オンデマンドでスケーリングできることです。これにより、それぞれが異なるハイパーパラメーター値を持つ実験の複数の実行を並行して処理するのに十分なコンピューティングリソースをプロビジョニングできます。

以前のラボで作成した** aml-cluster ** Azure Machine Learning計算クラスターを使用します（存在しない場合は作成されます）。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

try:
    # Get the cluster if it exists
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

## *ハイパードライブ*実験を実行する

Azure Machine Learningには、*Hyperdrive*実験によるハイパーパラメーター調整機能が含まれています。これらの実験は、それぞれが異なるハイパーパラメーターの組み合わせを持つ複数の子ランを起動します。最適化するログ記録されたターゲットパフォーマンスメトリックによって決定される最適なモデルを生成する実行を識別し、そのトレーニング済みモデルを登録および展開用に選択します。

In [ ]:
from azureml.core import Experiment
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails


# Sample a range of parameter values
params = GridParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--regularization': choice(0.001, 0.005, 0.01, 0.05, 0.1, 1.0)
    }
)


# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator that uses the remote compute
hyper_estimator = SKLearn(source_directory=experiment_folder,
                           inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the dataset as an input...
                           compute_target = training_cluster,
                           conda_packages=['pip==19.3.1'],
                           pip_packages=['azureml-sdk'], # ...so we need azureml-dataprep (it's in the SDK!)
                           entry_script='diabetes_training.py')

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(estimator=hyper_estimator, 
                          hyperparameter_sampling=params, 
                          policy=None, 
                          primary_metric_name='AUC', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=4)

# Run the experiment
experiment = Experiment(workspace = ws, name = 'diabates_training_hyperdrive')
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

上記のウィジェットで実験の実行ステータスを表示できます。 [Azure Machine Learning studio](https://ml.azure.com)でメインのHyperdrive実験の実行とその子の実行を表示することもできます。

> **Note**: ウィジェットにいくつかのエラーメッセージが表示され、数値以外のターゲットメトリックをプロットできないことが報告される場合があります。これらは無視できます！

## ベストパフォーマンスの実行を決定する

すべての実行が終了すると、指定したパフォーマンスメトリックに基づいて最適なものを見つけることができます（この場合、AUCが最適なもの）。

In [ ]:
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Regularization Rate:',parameter_values)

最良の実行が見つかったので、トレーニングしたモデルを登録できます。

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳しくは**: Hyperdriveの詳細については、[Azure MLドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters)を参照してください。